# Crawling

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import math
import re
import pandas as pd

In [2]:
def GetAllInfos():
    
    global SONG, ALBUM, DATE, GENRE, LIKECNT, COMMENTCNT, LYRIC, WRITER
        
    # HTML 파싱
    page = driver.page_source
    soup = BeautifulSoup(page, 'lxml')
        
    # 곡명, 앨범명, 발매일, 장르, 좋아요, 가사, 작사/작곡 수집
    SONG.append(soup.find(attrs={'class':'song_name'}).text.replace('\n', '').replace('\t', '').replace('곡명', ''))
    ALBUM.append(soup.find(attrs={'class':'meta'}).find_all('dd')[0].text)
    DATE.append(soup.find(attrs={'class':'meta'}).find_all('dd')[1].text)
    genres = soup.find(attrs={'class':'meta'}).find_all('dd')[2].text
    GENRE.append(genres.split(', '))
    LIKECNT.append(soup.find('span', attrs={'id':'d_like_count'}).text)
    COMMENTCNT.append(soup.find(attrs={'id':'revCnt'}).text)
    LYRIC.append(soup.find(attrs={'class':'lyric on'}).text.strip())
    writers = soup.find(attrs={'class':'list_person clfix'}).find_all(attrs={'class':'ellipsis artist'})
    WRITER.append(list(set([w.text.strip() for w in writers])))

In [3]:
# Settings
SONG = [] ; ALBUM = [] ; DATE = [] ; GENRE = [] ; LIKECNT = [] ; COMMENTCNT = [] ; LYRIC = [] ; WRITER = []
MELON_URL = 'https://www.melon.com/' ; QUERY = '아이유'

In [4]:
# 드라이버 생성 및 url 호출
options = webdriver.ChromeOptions()
options.add_argument('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15')
driver = webdriver.Chrome('chromedriver')
driver.get(MELON_URL)

In [5]:
# 아이유 검색
driver.find_element_by_xpath('//*[@id="top_search"]').clear()
driver.find_element_by_xpath('//*[@id="top_search"]').send_keys(QUERY)
driver.find_element_by_xpath('//*[@id="top_search"]').send_keys(Keys.RETURN)

In [6]:
time.sleep(5)
driver.find_element_by_xpath('//*[@id="divCollection"]/ul/li[3]/a').click() # 곡 탭 클릭
driver.find_element_by_xpath('//*[@id="conts"]/div[3]/div[1]/a[2]/span[1]').click() # 아티스트명에서 클릭

In [7]:
totalCnt = int(driver.find_element_by_xpath('//*[@id="conts"]/div[3]/h3/strong/em').text) # 총 건 수
pageNum = math.ceil(187/50) # 페이지 수

In [8]:
for page in range(1, pageNum+1):
    for i in range(1, 51):
        try:
            time.sleep(2)
            
            # 곡 상세 페이지로 이동
            driver.find_element_by_xpath('//*[@id="frm_defaultList"]/div/table/tbody/tr['+str(i)+']/td[3]/div/div/a[1]').click()
            # 펼치기 버튼 클릭
            driver.find_element_by_xpath('//*[@id="lyricArea"]/button').click()
    
            # 데이터 수집
            GetAllInfos()
        
            # 뒤로 가기
            driver.back()

        except Exception as e:
            print(page, i)
            print(e)
            
    if page<pageNum:
        driver.find_element_by_xpath('//*[@id="pageObjNavgation"]/div/span/a['+str(page)+']').click() # 다음 페이지

2 1
Message: element click intercepted: Element <a href="javascript:searchLog('web_song','SONG','SO','아이유','4711007');melon.link.goSongDetail('4711007');" title="곡정보 보기" class="btn btn_icon_detail">...</a> is not clickable at point (166, 625). Other element would receive the click: <html lang="ko" class="narrow-screen">...</html>
  (Session info: chrome=89.0.4389.114)

4 30
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="lyricArea"]/button"}
  (Session info: chrome=89.0.4389.114)

4 31
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="frm_defaultList"]/div/table/tbody/tr[31]/td[3]/div/div/a[1]"}
  (Session info: chrome=89.0.4389.114)

4 32
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="frm_defaultList"]/div/table/tbody/tr[32]/td[3]/div/div/a[1]"}
  (Session info: chrome=89.0.4389.114)

4 33
Message: no such element: Unable to locate element: {"method":"xpath",

In [9]:
# 드라이버 종료
driver.quit()

In [10]:
# 데이터프레임 생성
data = pd.DataFrame({'SongName':SONG, 'AlbumName':ALBUM, 'Date':DATE, 'Genre':GENRE, 'LikeCnt':LIKECNT, 'CommentCnt':COMMENTCNT, 'Lyrics':LYRIC, 'Writers':WRITER})
data

,SongName,AlbumName,Date,Genre,LikeCnt,CommentCnt,Lyrics,Writers
0,라일락,IU 5th Album 'LILAC',2021.03.25,[댄스],"154,396","1,717개",나리는 꽃가루에 눈이 따끔해 (아야)눈물이 고여도 꾹 참을래내 마음 한켠 비밀스런 ...,"[Dr.JO, 아이유, 웅킴, 임수호, N!ko]"
1,내 손을 잡아,최고의 사랑 OST Part.4,2011.05.25,"[발라드, 국내드라마]","130,792",959개,느낌이 오잖아 \n떨리고 있잖아\n언제까지 눈치만 볼 거니\n네 맘을 말해봐 \n딴...,"[G. 고릴라, 아이유]"
2,Celebrity,IU 5th Album 'LILAC',2021.03.25,[댄스],"269,327","3,790개","세상의 모서리구부정하게 커버린골칫거리 outsider걸음걸이, 옷차림, 이어폰 너머...","[아이유, Ryan S. Jhun, Jeppe London Bilsby, Lauri..."
3,Coin,IU 5th Album 'LILAC',2021.03.25,[R&B/Soul],"70,270","1,034개",강자에게 더 세게 I love gamble과감할수록 신세계 on my tableI’...,"[Kako, Poptime, 아이유]"
4,밤편지,밤편지,2017.03.24,[발라드],"408,922","2,411개",이 밤 그날의 반딧불을당신의 창 가까이 보낼게요음 사랑한다는 말이에요나 우리의 첫 ...,"[김희원, 제휘, 아이유]"
...,...,...,...,...,...,...,...,...
173,가여워 (Inst.),Growing Up,2009.04.23,[댄스],"8,978",3개,난 하루 종일 뭘 했는지 기억 안 나고난 손에 쥐고 있는 물건 찾으려 하고난 오늘이...,"[아이유와_유애나, 윤촤촤, 버터챙, 평민_]"
174,밤편지,Palette,2017.04.21,[발라드],"5,500",33개,이 밤 그날의 반딧불을 당신의 창 가까이 보낼게요음 사랑한다는 말 이에요나 우리의 ...,"[김희원, 제휘, 아이유]"
175,사랑이 잘 (With 오혁),Palette,2017.04.21,[R&B/Soul],"5,619",15개,미리 말할게 사과는 안 해아무 말 없이 너 후회 안 해다 관심 없잖아 친구야 뭐야O...,"[이종훈, 오혁, 아이유]"
176,봄 사랑 벚꽃 말고,HIGH4 1st Mini Album 'HI HIGH',2014.08.27,[발라드],"8,699",5개,길었던 겨우내 줄곧 품이 좀 남는밤색 코트그 속에 나를 쏙 감추고 걸음을재촉해 걸었...,"[이종훈, 이채규, 아이유]"


In [11]:
data.to_csv('../data/data.csv')